In [2]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import ascii, votable
from astropy.table import Table, vstack, hstack
import astropy.units as u
from astropy.coordinates import ICRS, Galactic
from scipy.interpolate import interp1d

from os import listdir

dir_rrlyrae = '/datascope/menard/group/scheng/Gaia/'

## load rrlyrae table

In [27]:
rrlyrae = np.load(dir_rrlyrae+'rrlyrae.npy')[0]['rrlyrae']
len(rrlyrae)

140784

## processing the name of light-curve folders

In [28]:
dir_list = listdir('/datascope/menard/DATA/Gaia/gdr2_new/Gaia/gdr2/light_curves/csv/')
id_list_start = np.zeros(len(dir_list),dtype=int)
id_list_end = np.zeros(len(dir_list),dtype=int)
for i in range(len(dir_list)):
    underscore = []
    dot = []
    for j in range(len(dir_list[i])):
        if dir_list[i][j] == '_':
            underscore.append(j)
        elif dir_list[i][j] == '.':
            dot.append(j)
    id_list_start[i] = dir_list[i][13:underscore[2]]
    id_list_end[i] = dir_list[i][underscore[2]+1:dot[0]]

## sort light curve folder by source id 

In [29]:
file_list = Table([id_list_end, dir_list],names=('id_list_end','dir_list'),dtype=(int,str))
file_list.add_column(Table.Column(np.arange(len(id_list_end)),name='index_unordered',dtype=int))
file_list.sort('id_list_end')
len(dir_list)

551

### sort RR Lyrae table by id

In [30]:
table_id_ordered = rrlyrae.copy()
table_id_ordered.add_column(Table.Column(np.arange(len(table_id_ordered)),name='index',dtype=int))
table_id_ordered.sort('source_id')
table_id_ordered.add_column(Table.Column(np.arange(len(table_id_ordered)),name='inverse_index',dtype=int))
table_id_ordered.sort('index')
inverse_index = table_id_ordered['inverse_index'].copy()
table_id_ordered.sort('source_id')

### select rr lyrae

In [31]:
selected_ab_18_bright = (table_id_ordered['best_classification']==b'RRab') * (table_id_ordered['int_average_g']<1000)
table_id_ordered_selected = table_id_ordered
len(table_id_ordered)

140784

# combine light-curve files & make index list for RR Lyrae

In [8]:
rrlyrae_index = np.load(dir_rrlyrae+'rrlyrae_lightcurve_index.npy')
light_curve_total = np.load(dir_rrlyrae+'light_curve_total.npy')[0]['light_curve_total']

In [36]:
import gc

light_curve_empty = Table.read('/datascope/menard/DATA/Gaia/gdr2_new/Gaia/gdr2/light_curves/csv/'+file_list['dir_list'][0],format = 'csv')
light_curve_empty.remove_rows(slice(0,len(light_curve_empty)))

light_curve_total = light_curve_empty
rrlyrae_index = np.zeros((len(rrlyrae),6),dtype=int) 
# start G, start BP, start RP, end RP, first_index_in_total_file, n

cursor_rrlyrae = 99292

for n_base in range(38,57):
    light_curve_total_temp = light_curve_empty
    gc.collect()
    for n in range(n_base*10+0,n_base*10+10): # for each light curve file
        print(n)
        light_curve_sample = Table.read('/datascope/menard/DATA/Gaia/gdr2_new/Gaia/gdr2/light_curves/csv/'+file_list['dir_list'][n],format = 'csv')  
        # open the file
        first_index_in_total_file = len(light_curve_total_temp) + len(light_curve_total)
                
        cursor_light_curve = 0  # read from the first line 
        source_id = table_id_ordered_selected['source_id'][cursor_rrlyrae]
        while source_id <= light_curve_sample['source_id'][-1]:
    
            while cursor_light_curve < len(light_curve_sample) and \
                    (light_curve_sample['source_id'][cursor_light_curve] < source_id):
                cursor_light_curve += 1 # find the first line for the cursor_rrlyrae
            start = cursor_light_curve
            
            while cursor_light_curve < len(light_curve_sample) and \
                    ( (light_curve_sample['source_id'][cursor_light_curve] == source_id) * (light_curve_sample['band'][cursor_light_curve] == 'G') ) :
                cursor_light_curve += 1 # find the last line for the cursor_rrlyrae
            start_BP = cursor_light_curve
            
            while cursor_light_curve < len(light_curve_sample) and \
                   ((light_curve_sample['source_id'][cursor_light_curve] == source_id) * (light_curve_sample['band'][cursor_light_curve] == 'BP') ):
                cursor_light_curve += 1 # find the last line for the cursor_rrlyrae
            start_RP = cursor_light_curve    
    
            while cursor_light_curve < len(light_curve_sample) and \
                  (  (light_curve_sample['source_id'][cursor_light_curve] == source_id) * (light_curve_sample['band'][cursor_light_curve] == 'RP') ):
                cursor_light_curve += 1 # find the last line for the cursor_rrlyrae
                if cursor_light_curve == len(light_curve_sample):
                    break
            end_RP = cursor_light_curve
            
            
            rrlyrae_index[cursor_rrlyrae,0] = int(start) # record the position of light curve for each rrlyrae
            rrlyrae_index[cursor_rrlyrae,1] = int(start_BP)
            rrlyrae_index[cursor_rrlyrae,2] = int(start_RP)
            rrlyrae_index[cursor_rrlyrae,3] = int(end_RP)
            rrlyrae_index[cursor_rrlyrae,4] = int(first_index_in_total_file)
            rrlyrae_index[cursor_rrlyrae,5] = int(n)
            
            cursor_rrlyrae += 1
            if cursor_rrlyrae >= len(table_id_ordered_selected):
                break
            source_id = table_id_ordered_selected['source_id'][cursor_rrlyrae]
        
        light_curve_total_temp = vstack((light_curve_total_temp,light_curve_sample)) # stack the lightcurves
        if cursor_rrlyrae >= len(table_id_ordered_selected):
            break
    
    light_curve_total = vstack((light_curve_total,light_curve_total_temp))
    if cursor_rrlyrae >= len(table_id_ordered_selected):
        break

380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550


### check the correctness

In [39]:
i=140783
print(table_id_ordered['source_id'][i],rrlyrae_index[i,5])
light_curve_total[rrlyrae_index[i,0]+rrlyrae_index[i,4]-1:rrlyrae_index[i,3]+rrlyrae_index[i,4]+1]

6917264079699449600 550


source_id,transit_id,band,time,mag,flux,flux_error,flux_over_error,rejected_by_photometry,rejected_by_variability,other_flags,solution_id
int64,int64,str2,float64,float64,float64,float64,float64,str5,str5,int64,int64
6917231987704428416,50768659788776230,RP,2314.9691984446677,17.442287449182217,846.9407235404029,39.43533255527813,21.476698,false,false,0,369295551293819386
6917264079699449600,20072450219857235,G,1759.8451417789568,19.268282010330086,369.8566961746517,14.856225030971387,24.895739,false,false,4097,369295549951641967
6917264079699449600,20076542511068047,G,1759.9191347005833,19.320326328347328,352.5459733652571,4.144063924094932,85.072525,false,false,524289,369295549951641967
6917264079699449600,21720777228980342,G,1789.651412334692,19.324673943641102,351.13709560502235,4.639141955544111,75.690094,false,false,1,369295549951641967
6917264079699449600,21730518037460592,G,1789.82755338045,18.952301959511043,494.79523229872694,8.265460523321343,59.862995,false,false,1,369295549951641967
6917264079699449600,22850334415418476,G,1810.0771105355218,19.33319209925581,348.39302396701777,8.288440888162961,42.033604,false,false,1,369295549951641967
6917264079699449600,22854426665738884,G,1810.1511110598385,19.293469431280137,361.37534353714665,6.447456292234253,56.04929,false,false,1,369295549951641967
6917264079699449600,29970608288838337,G,1938.845714957021,19.268910342548626,369.6427163462756,7.145272333245077,51.732487,false,false,1,369295549951641967
6917264079699449600,29974700574961931,G,1938.9197288011603,19.332786689390026,348.52313691836076,11.35059907619466,30.705263,false,false,1,369295549951641967


## save combined light-curve file & RR Lyrae index

In [40]:
np.save(dir_rrlyrae+'rrlyrae_lightcurve_index_380_plus.npy',rrlyrae_index)
np.save(dir_rrlyrae+'lightcurve_total_380_plus.npy',np.array([{'light_curve_total':light_curve_total}]))

## combine 380- and 380+

#### lightcurve

In [3]:
rrlyrae_index_plus = np.load(dir_rrlyrae+'rrlyrae_lightcurve_index_380_plus.npy')
light_curve_total_plus = np.load(dir_rrlyrae+'lightcurve_total_380_plus.npy')[0]['light_curve_total']

In [4]:
rrlyrae_index_minus = np.load(dir_rrlyrae+'rrlyrae_lightcurve_index_380_minus.npy')
light_curve_total_minus = np.load(dir_rrlyrae+'lightcurve_total_380_minus.npy')[0]['light_curve_total']

In [8]:
(light_curve_total_plus['band']=='G').sum()+(light_curve_total_minus['band']=='G').sum() == (light_curve_total['band']=='G').sum()

True

In [9]:
np.save(dir_rrlyrae+'lightcurve_total_380.npy',np.array([{'light_curve_total':light_curve_total}]))

#### index table

In [22]:
rrlyrae_index_plus[:,4] = rrlyrae_index_plus[:,4] + len(light_curve_total_minus)
rrlyrae_index_minus[99292:,:] = rrlyrae_index_plus[99292:,:]

In [33]:
rrlyrae_index = rrlyrae_index_minus

In [34]:
i=140783
print(table_id_ordered['source_id'][i],rrlyrae_index[i,5])
light_curve_total[rrlyrae_index[i,0]+rrlyrae_index[i,4]-1:rrlyrae_index[i,3]+rrlyrae_index[i,4]+1]

6917264079699449600 550


source_id,transit_id,band,time,mag,flux,flux_error,flux_over_error,rejected_by_photometry,rejected_by_variability,other_flags,solution_id
int64,int64,str2,float64,float64,float64,float64,float64,str5,str5,int64,int64
6917231987704428416,50768659788776230,RP,2314.9691984446677,17.442287449182217,846.9407235404029,39.43533255527813,21.476698,false,false,0,369295551293819386
6917264079699449600,20072450219857235,G,1759.8451417789568,19.268282010330086,369.8566961746517,14.856225030971387,24.895739,false,false,4097,369295549951641967
6917264079699449600,20076542511068047,G,1759.9191347005833,19.320326328347328,352.5459733652571,4.144063924094932,85.072525,false,false,524289,369295549951641967
6917264079699449600,21720777228980342,G,1789.651412334692,19.324673943641102,351.13709560502235,4.639141955544111,75.690094,false,false,1,369295549951641967
6917264079699449600,21730518037460592,G,1789.82755338045,18.952301959511043,494.79523229872694,8.265460523321343,59.862995,false,false,1,369295549951641967
6917264079699449600,22850334415418476,G,1810.0771105355218,19.33319209925581,348.39302396701777,8.288440888162961,42.033604,false,false,1,369295549951641967
6917264079699449600,22854426665738884,G,1810.1511110598385,19.293469431280137,361.37534353714665,6.447456292234253,56.04929,false,false,1,369295549951641967
6917264079699449600,29970608288838337,G,1938.845714957021,19.268910342548626,369.6427163462756,7.145272333245077,51.732487,false,false,1,369295549951641967
6917264079699449600,29974700574961931,G,1938.9197288011603,19.332786689390026,348.52313691836076,11.35059907619466,30.705263,false,false,1,369295549951641967


In [37]:
len(rrlyrae_index)

140784

In [35]:
np.save(dir_rrlyrae+'rrlyrae_lightcurve_index.npy',np.array([{'rrlyrae_index':rrlyrae_index}]))